In [55]:
import string, sys
import nltk
import Stemmer
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *

STOPWORDS = set(stopwords.words('english')) 
URL_STOP_WORDS = set(["http", "https", "www", "ftp", "com", "net", "org", "archives", "pdf", "html", "png", "txt", "redirect", "align", "realign", "valign", "nonalign", "malign", "unalign", "salign", "qalign", "halign", "font", "fontsiz", "fontcolor", "backgroundcolor", "background", "style", "center", "text"])
fields= ['t:', 'i:', 'c:', 'b:', 'r:', 'l:']

In [56]:
def readIndex():
    f= open("./InvertedIndices/InvertedIndex.txt", "r")
    Index= {}

    lines= f.readlines()
    line =0

    while line< len(lines):
        text = lines[line].split(":")
        word= text[0]
        frequency= text[1]
        line +=1
        while line< len(lines) and ":" not in lines[line]:
            frequency += lines[line]
            line +=1
        Index[word]= frequency

    f.close()
    return Index

In [57]:
def cleanQuery(query):
    stemmer= Stemmer.Stemmer("english")
    query = re.sub(r'\\d+', '', query) #Remove numbers
    query = re.sub(r'<(.*?)>','',query) #Remove tags if any
    query = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', query, flags=re.MULTILINE) #Remove Url
    query = re.sub(r'{\|(.*?)\|}', '', query, flags=re.MULTILINE) #Remove CSS
    query = re.sub(r'\[\[file:(.*?)\]\]', '', query, flags=re.MULTILINE) #Remove File
    query = re.sub(r'[^\w\s]', '', query) 
    query = query.split()
    query = [x for x in query if x not in STOPWORDS and x not in URL_STOP_WORDS]
    query = [stemmer.stemWord(word) for word in query]
#     print ("Query in Processing: {}".format(query))
    
    return query

In [58]:
def search_fieldQuery(documents, field_type):
    fieldDocs= []
    documents= documents.split("|")
    for doc in documents:
        if field_type in doc:
            fieldDocs.append(doc)
    return fieldDocs

In [59]:
def process_fieldQuery(query):
    fieldInfo= {}
    
    for f in fields:
        field= query.find(f)
        if field !=-1:
            fieldInfo[field]= f
    
    fieldInfo= sorted(fieldInfo.items())
    fieldInfo.append((1234567890, ""))
    print(fieldInfo)
    i=0
    while i+1 <len(fieldInfo):
        fieldQuery = (query[fieldInfo[i][0]+2 : fieldInfo[i+1][0]]).lower()
#         print (fieldQuery)
        fieldQuery = cleanQuery(fieldQuery)
#         print (fieldQuery)
        for word in fieldQuery:
            if word not in Index:
                print (word, " : ", [])
            else:
                value= Index[word]
                print(word, " : ", search_fieldQuery(value, fieldInfo[i][1][:1]))
            print()
        i +=1
    

In [60]:
Index= readIndex()

query= "t:World Cup i:2019 c:Cricket"

if "t:" in query or "i:" in query or "c:" in query or "b:"in query or "r:" in query or "l:" in query:    
    process_fieldQuery(query)
else:
    query = cleanQuery(query)
    for word in query:
        if word.lower() in Index:
            print (word, " : ", Index[word.lower()])
        else:
            print (word, " : ", [])

[(0, 't:'), (12, 'i:'), (19, 'c:'), (1234567890, '')]
world  :  ['1166-t1c4b31', '2229-t1i3c1b71', '2230-t1i1c2b23', '2984-t1i1c2b393', '3060-t1i3c1b79l15', '3213-t1', '3617-t1b8', '4418-t1i19c4b213l1', '5864-t1', '5866-t1', '6411-t1c1b1', '7151-t1i29c2b399l1', '7318-t1c2b11', '7763-t1i3c1b81', '7774-t1i2b15', '8792-t1', '9068-t1', '9442-t1b8', '10443-t1', '10464-t1i4c1b45', '12386-t1b2r1', '14046-t1b50', '14829-t1i1b7l1', '16648-t1i18c4b199l1', '16860-t1b49', '17070-t1', '18367-t1c1b1', '19482-t1b59']

cup  :  ['4418-t1i3c4b125l1', '7058-t1i4c3b499l4', '7151-t1i3c1b245l1', '10464-t1i4c1b39', '13280-t1i5c1b181l10', '16648-t1i7c2b74', '17070-t1']

2019  :  ['12-i1b3', '66-i1b7', '74-i5b48', '86-i4b9', '105-i3b31', '108-i5b1', '139-i2b34', '140-i2', '160-i8', '161-i3b23', '168-i2b8', '171-i20', '173-i2', '177-i1b14', '185-i26', '186-i2b19', '190-i5b8', '203-i5b1', '254-i9b50', '262-i2', '281-i1', '284-i4', '303-i1', '322-i3b14', '396-i21', '406-i64', '421-i1b11', '430-i1b26', '474-i16', 